# Generating counterfactual explanations with any ML model

The goal of this notebook is to show how to generate CFs for ML models using frameworks other than TensorFlow or PyTorch. We show how to generate diverse CFs by three methods: 
1. Independent random sampling of features (`method_name='random')`
2. Genetic algorithm (`method_name='genetic')`
3. Querying a KD tree (`method_name='kdtree')`

We use scikit-learn models for demonstration.  

## 1. Independent random sampling of features

In [1]:
# import DiCE
import dice_ml
from dice_ml.utils import helpers  # helper functions

from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.ensemble import RandomForestClassifier

In [3]:
%load_ext autoreload
%autoreload 2

### Loading dataset

We use the "adult" income dataset from UCI Machine Learning Repository (https://archive.ics.uci.edu/ml/datasets/adult). We transform the data as described in dice_ml.utils.helpers module.

In [13]:
from data_utils import get_model, read_dataset

ds = read_dataset()
dataset = pd.concat([df.assign(project=p, split=s) for p,(tr,te) in projects.items() for s,df in (('train', tr), ('test', te))], ignore_index=True)



In [9]:
dataset.head()

,CountDeclMethodPrivate,CountDeclMethodDefault,AvgEssential,CountDeclClassVariable,CountDeclClassMethod,AvgLineComment,CountLineComment,CountDeclClass,CountDeclMethodProtected,CountDeclInstanceVariable,...,AvgCyclomaticModified,AvgLineBlank,CountDeclInstanceMethod,CountOutput_Mean,MINOR_LINE,CountPath_Min,MAJOR_COMMIT,Del_lines,ADEV,AvgCyclomaticStrict
0,43.0,0.0,1,36.0,1.0,3,394.0,3.0,0.0,13.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0.0,0.0,5,1.0,1.0,4,91.0,1.0,8.0,27.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3.0,0.0,1,0.0,1.0,3,70.0,1.0,0.0,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2.0,0.0,1,12.0,1.0,2,78.0,1.0,0.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0.0,0.0,1,0.0,1.0,4,61.0,1.0,0.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
from sklearn.preprocessing import StandardScaler
class ScaledModel:
    """
    Wraps an sklearn-like classifier so it accepts *unscaled* X and internally
    applies a StandardScaler (fit on train features).
    Provides predict and predict_proba for Dice/LIME verification.
    """

    def __init__(self, base_model, scaler: StandardScaler):
        self.model = base_model
        self.scaler = scaler
        if hasattr(base_model, "classes_"):
            self.classes_ = base_model.classes_

    def predict(self, X):
        Xs = self.scaler.transform(X)
        return self.model.predict(Xs)

    def predict_proba(self, X):
        Xs = self.scaler.transform(X)
        if hasattr(self.model, "predict_proba"):
            proba = self.model.predict_proba(Xs)
            if proba.ndim == 2 and proba.shape[1] == 2:
                return proba
            if proba.ndim == 2 and proba.shape[1] == 1:
                p1 = proba[:, 0]
                return np.stack([1.0 - p1, p1], axis=1)
            if proba.ndim == 1:
                p1 = proba
                return np.stack([1.0 - p1, p1], axis=1)
        if hasattr(self.model, "decision_function"):
            s = self.model.decision_function(Xs)
            s = np.clip(s, -50, 50)
            p1 = 1.0 / (1.0 + np.exp(-s))
            if p1.ndim == 1:
                return np.stack([1.0 - p1, p1], axis=1)
            p1r = p1[:, 0]
            return np.stack([1.0 - p1r, p1r], axis=1)
        y = self.model.predict(Xs)
        p0 = (y == 0).astype(float)
        return np.stack([p0, 1.0 - p0], axis=1)

    def __getattr__(self, name):
        return getattr(self.model, name)
train, test = ds["activemq@0"]
feat_cols = [c for c in test.columns if c != "target"]
base_model = get_model("activemq@0", "RandomForest")
scaler = StandardScaler().fit(train[feat_cols].values)
model = ScaledModel(base_model, scaler)
d = dice_ml.Data(dataframe=dataset, continuous_features=feat_cols, outcome_name='target')

### Training a custom ML model

Below, we build an ML model using scikit-learn to demonstrate how our methods can work with any sklearn model. 

In [ ]:
target = dataset["target"]
# Split data into train and test
datasetX = dataset.drop("target", axis=1)
x_train, x_test, y_train, y_test = train_test_split(datasetX,
                                                    target,
                                                    test_size=0.2,
                                                    random_state=0,
                                                    stratify=target)

numerical = feat_cols

categorical = x_train.columns.difference(numerical)

# We create the preprocessing pipelines for both numeric and categorical data.
numeric_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())])

categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

transformations = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numerical),
        ('cat', categorical_transformer, categorical)])

# Append classifier to preprocessing pipeline.
# Now we have a full prediction pipeline.
clf = Pipeline(steps=[('preprocessor', transformations),
                      ('classifier', RandomForestClassifier())])
model = clf.fit(x_train, y_train)

In [ ]:
# provide the trained ML model to DiCE's model object
backend = 'sklearn'
m = dice_ml.Model(model=model, backend=backend)

### Generate diverse counterfactuals

In [ ]:
# initiate DiCE
exp_random = dice_ml.Dice(d, m, method="random")

In [ ]:
query_instances = x_train[4:6]

In [ ]:
# generate counterfactuals
dice_exp_random = exp_random.generate_counterfactuals(query_instances, total_CFs=2, desired_class="opposite", verbose=False)


In [ ]:
dice_exp_random.visualize_as_dataframe(show_only_changes=True)

It can be observed that the random sampling method produces less sparse CFs in contrast to current DiCE's implementation. The sparsity issue with random sampling worsens with increasing *total_CFs* 

Further, different sets of counterfactuals can be generated with different random seeds.

In [ ]:
# generate counterfactuals
# default random seed is 17
dice_exp_random = exp_random.generate_counterfactuals(query_instances,
                                                      total_CFs=4,
                                                      desired_class="opposite",
                                                      random_seed=9)

In [ ]:
dice_exp_random.visualize_as_dataframe(show_only_changes=True)

#### Selecting the features to vary

Here, you can ensure that DiCE varies only features that it makes sense to vary.

In [ ]:
# generate counterfactuals
dice_exp_random = exp_random.generate_counterfactuals(
        query_instances, total_CFs=4, desired_class="opposite",
        features_to_vary=['workclass', 'education', 'occupation', 'hours_per_week'])

In [ ]:
dice_exp_random.visualize_as_dataframe(show_only_changes=True)

#### Choosing feature ranges

Since the features are sampled randomly, they can freely vary across their range. In the below example, we show how range of continuous features can be controlled using *permitted_range* parameter that can now be passed during CF generation.

In [ ]:
# generate counterfactuals
dice_exp_random = exp_random.generate_counterfactuals(
    query_instances, total_CFs=4, desired_class="opposite",
    permitted_range={'age': [22, 50], 'hours_per_week': [40, 60]})

In [ ]:
dice_exp_random.visualize_as_dataframe(show_only_changes=True)

## 2. Genetic Algorithm

Here, we show how to use DiCE can be used to generate CFs for any ML model by using the genetic algorithm to find the best counterfactuals close to the query point. The genetic algorithm converges quickly, and promotes diverse counterfactuals. 

### Training a custom ML model

Currently, the genetic algorithm method works with scikit-learn models. We will use the same model as shown previously in the notebook. Support for Tensorflow 1&2 and Pytorch will be implemented soon.

### Generate diverse counterfactuals

In [ ]:
# initiate DiceGenetic
exp_genetic = dice_ml.Dice(d, m, method='genetic')

In [ ]:
# generate counterfactuals
dice_exp_genetic = exp_genetic.generate_counterfactuals(query_instances, total_CFs=4, desired_class="opposite", verbose=True)

In [ ]:
dice_exp_genetic.visualize_as_dataframe(show_only_changes=True)

We can also ensure that the genetic algorithm also only varies the features that you wish to vary

In [ ]:
# generate counterfactuals
dice_exp_genetic = exp_genetic.generate_counterfactuals(
    query_instances, total_CFs=2, desired_class="opposite",
    features_to_vary=['workclass', 'education', 'occupation', 'hours_per_week'])
dice_exp_genetic.visualize_as_dataframe(show_only_changes=True)

You can also constrain the features to vary only within the permitted range

In [ ]:
# generate counterfactuals
dice_exp_genetic = exp_genetic.generate_counterfactuals(
    query_instances, total_CFs=2, desired_class="opposite",
    permitted_range={'age': [22, 50], 'hours_per_week': [40, 60]})
dice_exp_genetic.visualize_as_dataframe(show_only_changes=True)

## 3. Querying a KD Tree

Here, we show how to use DiCE can be used to generate CFs for any ML model by finding the closest points in the dataset that give the output as the desired class. We do this efficiently by building KD trees for each class, and querying the KD tree of the desired class to find the k closest counterfactuals from the dataset. The idea behind finding the closest points from the training data itself is to ensure that the counterfactuals displayed are feasible.

### Training a custom ML model

Currently, the KD tree algorithm method works with scikit-learn models. Again, we will use the same model as shown previously in the notebook. Support for Tensorflow 1&2 and Pytorch will be implemented soon.

### Generate diverse counterfactuals

In [ ]:
# initiate DiceKD
exp_KD = dice_ml.Dice(d, m, method='kdtree')

In [ ]:
# generate counterfactuals
dice_exp_KD = exp_KD.generate_counterfactuals(query_instances, total_CFs=4, desired_class="opposite")

In [ ]:
dice_exp_KD.visualize_as_dataframe(show_only_changes=True)

#### Selecting the features to vary

Here, again, you can vary only features that you wish to vary. Please note that the output counterfactuals are only from the training data. If you want other counterfactuals, please use the random or genetic method. 

In [ ]:
# generate counterfactuals
dice_exp_KD = exp_KD.generate_counterfactuals(
    query_instances, total_CFs=4, desired_class="opposite",
    features_to_vary=['age', 'workclass', 'education', 'occupation', 'hours_per_week'])

In [ ]:
dice_exp_KD.visualize_as_dataframe(show_only_changes=True)

#### Selecting the feature ranges

Here, you can control the ranges of continuous features. 

In [ ]:
# generate counterfactuals
dice_exp_KD = exp_KD.generate_counterfactuals(
    query_instances, total_CFs=5, desired_class="opposite",
    permitted_range={'age': [30, 50], 'hours_per_week': [40, 60]})
dice_exp_KD.visualize_as_dataframe(show_only_changes=True)